In [1]:
import pandas as pd
import re
from math import sin, asin, cos, radians, fabs, sqrt

In [1]:
classfy = {
    'track': ['track glide','fitness track'],
     
    'area': ['synthetic surface','quiet areas and active play areas', 'grassed areas', 'sports field'],
     
    'walking': ['boardwalk', 'rubber pathways'],
     
    'basic facilities': ['car parking', 'public toilet with change table', 'disabled access',
  'some disabled access',
  'play panel and small stage area some disabled access','steps', 'terracotta cats to pat','some shade', 'shade structure', 'lots of shade', 'shade','sheep sculptures','stockyard style fencing','barbecue', 'barbecues', 'barbeque', 'public toilet','picnic table','picnic tables','shop front', 'picnic settings', 'drinking fountain', 'seats'],
     
    'carousel': ['carousels', 'carousel and slide', 'carousel'],
     
    'integrated': ['water play area including a water pump and water channel along the bluestone paving', 'climbing,swinging and balancing elements suspended from timber poles', 'tall tyre swing and tyre spring spinners', 'rocky escarpment with slides,swings and water play', 'combination unit','multi-level play structure','pirate ship play structure','combination units','cubby', 'cubby with slide','fort', 'fort tower', 'small tower', 'wooden tower','platform', 'platforms', 'tower platforms', 'waterplay zone – perfect in summer!', 'rocky escarpment with slides,swings and water play'],
     
    'rocker': ['tall tyre swing and tyre spring spinners', 'spinners', 'spinner','spring rocker', 'rockers', 'rocker', '4-way spring rocker','rocking table','spring rider', 'spring riders'],
     
    'sand': ['sandplay items', 'excavator', 'extensive sandpit with a large digger', 'sandplay toy', 'sandpits', 'sandpit with diggers', 'sandpit', 'sand pit'],
     
    'see-saw': ['see-saw', 'see saw', 'see saws'],
     
    'slide': ['cubby-like structures with climbing and sliding challenges',
  'slide',
  'double slide',
  'spiral slide',
  'carousel and slide',
  'toddler playspace - slide',
  'slides',
  'cubby with slide',
  'slide and climbing features',
  'dragon slide',
  'cubby-like structures with climbing and sliding challenges', 'rocky escarpment with slides,swings and water play'],
     
    'swing': ['climbing,swinging and balancing elements suspended from timber poles', 'tall tyre swing and tyre spring spinners', 'swings',
  'swing',
  '5-way swing',
  'swinging hammocks',
  'liberty swing for wheelchairs',
  '1 x 5 way swing',
  'rocking swing',
  'sandplit and swings',
  'giraffe swing', 'rocky escarpment with slides,swings and water play'],
     
    'climb': ['climbing,swinging and balancing elements suspended from timber poles', 'climbing forest', 
  'cubby-like structures with climbing and sliding challenges',
  'climbers',
  'climbers x 3',
  'climber',
  'rock and rope climbing features',
  'climbing frame',
  'climbing features',
  'rock wall', 'rocky escarpment with slides,swings and water play'],
     
    'cool': ['concrete skate ramp', 'skate park','basketball court',
  'play court space with basketball backboard only',
  'temporary basketball court with backboard and ring',
  'basketball rings','bmx track'],
     
    'bar': ['chin up bar', 'chin up bars', 'monkey bars', 'parallel bars'],
     
    'balance': ['climbing,swinging and balancing elements suspended from timber poles', 'rope bridge', 'balancing bridges','balance beam', 'stepping posts'],
     
    'mini-game': ['tunnels and barrels', 'tunnels', 'play units with mirror panel','tic-tac-toe', 'abacus', 'chalkboard','clock and steering wheel','musical wall','bells','musical elements', 'speaking tubes']}

In [46]:
mel_all = pd.read_csv('mel.csv')
mel_all = mel_all.drop(columns = ['council_re','location_d'])

In [23]:
mel_all[mel_all['features'].isnull()]

,the_geom,name,features


In [25]:
mel_all.loc[42,'features'] = 'Swings, slide, rock wall, monkey bars, carousel, see saw, sand pit, tunnels and barrels, Temporary basketball court with backboard and ring.'

In [47]:
equipment = list(mel_all['features'])
unique_equipment = {}
number = []
for i in equipment:
    index = equipment.index(i)
    if type(i) == str :
        temp = i.replace('.','').lower().split(', ')
        number.append(len(temp))
        for j in temp:
            for k,v in classfy.items() :
                if j in v :
                    if k not in unique_equipment :
                        unique_equipment[k] = []
                        unique_equipment[k].append(index)
                    else :
                        unique_equipment[k].append(index)
    else :
        number.append(0)
                    

In [48]:
mel_all['total'] = number
for k,v in unique_equipment.items() :
    temp = [0] * len(equipment)
    for each in v :
        temp[each] = 1
    mel_all[k] = temp


In [49]:
sensor_all = pd.read_csv('Pedestrian_Counting_System_-_Sensor_Locations.csv')
sensor_all.head()

,sensor_id,sensor_description,sensor_name,installation_date,status,note,direction_1,direction_2,latitude,longitude,location
0,59,Building 80 RMIT,RMIT_T,2019/02/13,A,NaN,North,South,-37.808256,144.963049,"(-37.80825648, 144.96304859)"
1,23,Spencer St-Collins St (South),Col623_T,2013/09/02,A,NaN,East,West,-37.819093,144.954527,"(-37.81909256, 144.95452749)"
2,20,Chinatown-Lt Bourke St (South),LtB170_T,2013/09/06,A,NaN,East,West,-37.811729,144.968247,"(-37.81172913, 144.9682466)"
3,34,Flinders St-Spark La,Fli32_T,2014/06/08,A,NaN,East,West,-37.815380,144.974150,"(-37.81537985, 144.9741505)"
4,57,Bourke St Bridge,BouBri_T,2018/08/13,A,NaN,West,East,-37.817673,144.950256,"(-37.8176735, 144.95025595)"


In [50]:
geom_lat = []
geom_lon = []
geom_point = []
lon_info = r'\d\d\d\.\d+'
lat_info = r'\-\d+\.\d+'
for i in range(len(equipment)) : 
    lat_list = list(map(float,re.findall(lat_info,list(mel_all['the_geom'])[i])))
    lon_list = list(map(float,re.findall(lon_info,list(mel_all['the_geom'])[i])))
    lat = sum(lat_list)/len(lat_list)
    lon = sum(lon_list)/len(lon_list)
    geom_lat.append(lat)
    geom_lon.append(lon)
    geom_point.append('POINT ('+str(lon)+','+str(lat)+')')

In [51]:
sensor_id = list(sensor_all['sensor_id'])
sensor_lat = list(sensor_all['latitude'])
sensor_lon = list(sensor_all['longitude'])

In [52]:
def distance(lon1, lat1, lon2, lat2):
    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
    r = 6378 * 1000
    a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
    c = 2*asin(sqrt(a))
    result = r*c
    return result
distance_list = []
lowest_id = []
for index in range(len(geom_point)):
    temp = None
    temp_id = None
    for si in range(len(sensor_id)):
        d = distance(geom_lon[index],geom_lat[index],sensor_lon[si],sensor_lat[si])
        if temp == None or temp > d:
            temp = d
            temp_id = sensor_id[si]
    distance_list.append(temp)
    lowest_id.append(temp_id)

In [53]:
mel_all['location'] = geom_point
mel_all['distance'] = distance_list
mel_all['sensor'] = lowest_id
mel_all = mel_all.drop(columns = ['the_geom'])

In [54]:
list(mel_all[mel_all.distance < 200].name.value_counts().keys())

['Lincoln Square Playground',
 'Docklands Park Playground',
 'Birrarung Marr Artplay']

In [62]:
two_sensor = [8,46,7]

In [39]:
feature_df = mel_all[mel_all['name'].isin(two)]

In [44]:
feature_df = feature_df.drop(columns = ['distance','sensor','location','name','features'])
feature_df

,total,bar,balance,basic facilities,cool,rocker,swing,integrated,slide,see-saw,climb,mini-game,carousel,sand,walking,track,area
6,5,0,0,1,0,0,1,1,1,0,0,0,1,0,0,0,0
9,6,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0
12,8,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1


In [63]:
counts_df = pd.read_csv('Pedestrian_Counting_System___2009_to_Present__counts_per_hour_.csv')
counts_df.head()

,ID,Date_Time,Year,Month,Mdate,Day,Time,Sensor_ID,Sensor_Name,Hourly_Counts
0,2887628,11/01/2019 05:00:00 PM,2019,November,1,Friday,17,34,Flinders St-Spark La,300
1,2887629,11/01/2019 05:00:00 PM,2019,November,1,Friday,17,39,Alfred Place,604
2,2887630,11/01/2019 05:00:00 PM,2019,November,1,Friday,17,37,Lygon St (East),216
3,2887631,11/01/2019 05:00:00 PM,2019,November,1,Friday,17,40,Lonsdale St-Spring St (West),627
4,2887632,11/01/2019 05:00:00 PM,2019,November,1,Friday,17,36,Queen St (West),774


In [64]:
c_list = []
for i in two_sensor:
    temp = counts_df[counts_df['Sensor_ID']==i]
    cc = list(temp['Hourly_Counts'])
    c = sum(cc)/len(cc)*24
    c_list.append(c)

In [66]:
feature_df['count'] = c_list
feature_df.to_csv('feature.csv',index=False)

In [67]:
other = pd.read_csv('./Brisbane/feature.csv')

In [73]:
merged = pd.concat([feature_df,other])

In [72]:
feature_df

,total,bar,balance,basic facilities,cool,rocker,swing,integrated,slide,see-saw,climb,mini-game,carousel,sand,walking,track,area,count
6,5,0,0,1,0,0,1,1,1,0,0,0,1,0,0,0,0,3611.007309
9,6,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,2251.797196
12,8,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,8608.074477


In [75]:
other

,total,basic facilities,area,cool,integrated,walking,rocker,see-saw,slide,climb,bar,mini-game,swing,balance,sand,track,carousel,count
0,32,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,228.427397
1,16,1,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,228.427397
2,49,1,1,1,1,1,1,1,0,1,1,0,0,0,0,0,0,453.250000
3,546,1,1,1,1,1,1,1,1,0,1,1,1,0,0,0,0,1435.750000
4,21,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1486.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5591.250000
71,93,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,3532.000000
72,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2914.250000
73,14,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2439.000000


In [76]:
merged.to_csv('ready.csv')

In [2]:
list(classfy.keys())

['track',
 'area',
 'walking',
 'basic facilities',
 'carousel',
 'integrated',
 'rocker',
 'sand',
 'see-saw',
 'slide',
 'swing',
 'climb',
 'cool',
 'bar',
 'balance',
 'mini-game']